In [ ]:
debug = False

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

# from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.agents import AgentType, AgentExecutor
from langchain.tools import StructuredTool
from langchain.chat_models import ChatOpenAI

from typing import List, Dict
import panel as pn
import param
import pandas as pd
import geopandas as gpd

from modules.chat_tools import tools, chat_box
# from modules.rasterize_plots import s2_hv_plot, create_rgb_viewer

pd.options.plotting.backend = 'holoviews'

In [ ]:
# this is just my little agent & panel layout/ template

def chat(user_messages: List[Dict[str, str]]) -> None:
    # user_messages = [{"You": "Your input"}, {"AI": "A response"}, ...]
    user_message = user_messages[-1]
    input = user_message.get("You")
    if input is None:
        return
    chat_box.append({"SatGPT": agent.run(input=input)})

pn.bind(chat, user_messages=chat_box, watch=True)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)


agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True,
    agent_kwargs=agent_kwargs, 
    memory=memory
    )
component = pn.Column(chat_box, height=800)

template = pn.template.FastListTemplate(
    # site="Awesome Panel",
    title="SatGPT",
    logo="https://panel.holoviz.org/_static/logo_stacked.png",
    main=[component],
)

if debug == False:
    template.servable()


In [ ]:
if debug == True:
    pn.serve(template)

In [ ]:
# agent.run("search sentinel images over seattle, May 2022 & load to map")

template = pn.template.FastListTemplate(
    # site="Awesome Panel",
    title="SatGPT",
    logo="https://panel.holoviz.org/_static/logo_stacked.png",
    header_background=ACCENT_BASE_COLOR,
    accent_base_color=ACCENT_BASE_COLOR,
    main=[component],
).servable()